# Citibike ML
In this example we use the [Citibike dataset](https://ride.citibikenyc.com/system-data). Citibike is a bicycle sharing system in New York City. Everyday users choose from 20,000 bicycles at 1300 stations around New York City.

To ensure customer satisfaction Citibike needs to predict how many bicycles will be needed at each station. Maintenance teams from Citibike will check each station and repair or replace bicycles. Additionally, the team will relocate bicycles between stations based on predicted demand. The business needs to be able to run reports of how many bicycles will be needed at a given station on a given day.

## ML Engineering Development
In this section of the demo, we will utilize Snowpark's Python client-side Dataframe API to build and develope code for the **ML Ops pipeline**.  We will take the functions and model training/inference definition from the data scientist and put it into production using the Snowpark server-side runtime and Snowpark Python user-defined functions for ML model training and inference.

The ML Engineer will start by exploring the deoployment options and testing the deployed model before building a pipeline.

For this demo flow we will assume that the organization has the following **policies and processes** :   
-**Dev Tools**: The ML engineer can develop in their tool of choice (ie. VS Code, IntelliJ, Pycharm, Eclipse, etc.).  Snowpark Python makes it possible to use any environment where they have a python kernel.  For the sake of a demo we will use Jupyter.  
-**Data Governance**: To preserve customer privacy no data can be stored locally.  The ingest system may store data temporarily but it must be assumed that, in production, the ingest system will not preserve intermediate data products between runs. Snowpark Python allows the user to push-down all operations to Snowflake and bring the code to the data.   
-**Automation**: Although the ML engineer can use any IDE or notebooks for development purposes the final product must be python code at the end of the work stream.  Well-documented, modularized code is necessary for good ML operations and to interface with the company's CI/CD and orchestration tools.  
-**Compliance**: Any ML models must be traceable back to the original data set used for training.  The business needs to be able to easily remove specific user data from training datasets and retrain models. 

Input: Data in `trips` table.  Feature engineering, train, predict functions from data scientist.  
Output: Prediction models available to business users in SQL. Evaluation reports for monitoring.

### 1. Load  credentials and connect to Snowflake

In [ ]:
from dags.snowpark_connection import snowpark_connect
session, state_dict = snowpark_connect('./include/state.json')

### 2.  Create Feature Pipelines


In [ ]:
trips_table_name = state_dict['trips_table_name']
holiday_table_name = state_dict['holiday_table_name']
weather_table_name = state_dict['weather_table_name']

We will materialize the holiday and weather datasets as tables instead of calculating each time in the inference and training pipelines.

In [ ]:
import snowflake.snowpark as snp
from snowflake.snowpark import functions as F 
from dags.feature_engineering import generate_holiday_df, generate_weather_df

holiday_df = generate_holiday_df(session, holiday_table_name) 
holiday_df.write.mode('overwrite').saveAsTable(holiday_table_name)

weather_df = generate_weather_df(session, weather_table_name) 
weather_df.write.mode('overwrite').saveAsTable(weather_table_name)


We will need to forecast features for weather (TEMP) and also holidays.  For weather we could use forecast from the Snowflake Marketplace providers like Weather Source.  Since we went back in time for this hands-on-lab the weather "forecast" is actually in the historical weather tables.

In [ ]:
def create_forecast_df(session, holiday_table_name:str, weather_table_name:str, start_date:str, steps:int):
    from dags.feature_engineering import generate_holiday_df, generate_weather_df
    from datetime import timedelta, datetime
    #import snowflake.snowpark as snp
    from snowflake.snowpark import functions as F 
    
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = start_date+timedelta(days=steps)

    #check if it tables already materialized, otherwise generate DF
    holiday_df = session.table(holiday_table_name)
    try: 
        _ = holiday_df.columns()
    except:
        holiday_df = generate_holiday_df(session, holiday_table_name)
        
    weather_df = session.table(weather_table_name)[['DATE','TEMP']]
    try: 
        _ = weather_df.columns()
    except:
        weather_df = generate_weather_df(session, weather_table_name)[['DATE','TEMP']]
        
    forecast_df = holiday_df.join(weather_df, 'DATE', join_type='right')\
                            .na.fill({'HOLIDAY':0})\
                            .filter((F.col('DATE') >= start_date) &\
                                    (F.col('DATE') <= end_date))\
                            .sort('DATE', ascending=True)
    return forecast_df

In [ ]:
forecast_df = create_forecast_df(session=session, 
                                 holiday_table_name=state_dict['holiday_table_name'], 
                                 weather_table_name=state_dict['weather_table_name'], 
                                 start_date='2020-03-01', 
                                 steps=30)
forecast_df.show()

### 3. Create UDF for Training and Inference

Since this is a time series prediction we will retrain a model each time we do inference.  We don't need to save the model artefacts but we will save the predictions in an predictions table.  
  
Here we can use Snowpark User Defined Functions for training as well as inference without having to pull data out of Snowflake.

In [ ]:
%%writefile dags/station_train_predict.py

def station_train_predict_func(historical_data:list, 
                               historical_column_names:list, 
                               target_column:str,
                               cutpoint: int, 
                               max_epochs: int, 
                               forecast_data:list,
                               forecast_column_names:list,
                               lag_values:list) -> str:
    
    from torch import tensor
    import pandas as pd
    from pytorch_tabnet.tab_model import TabNetRegressor
    from datetime import timedelta
    import numpy as np
    
    feature_columns = historical_column_names.copy()
    feature_columns.remove('DATE')
    feature_columns.remove('STATION_ID')
    feature_columns.remove(target_column)
    
    df = pd.DataFrame(historical_data, columns = historical_column_names)
    #df['DATE']=pd.to_datetime(df['DATE']).dt.date
    
    y_valid = df[target_column][-cutpoint:].values.reshape(-1, 1)
    X_valid = df[feature_columns][-cutpoint:].values
    y_train = df[target_column][:-cutpoint].values.reshape(-1, 1)
    X_train = df[feature_columns][:-cutpoint].values
    
    batch_df = pd.DataFrame(range(2,65,2), columns=['batch_size'])
    batch_df['batch_remainder'] = len(X_train)%batch_df['batch_size']
    optimal_batch_size=int(batch_df['batch_size'].where(batch_df['batch_remainder']==batch_df['batch_remainder'].min()).max())
    
    model = TabNetRegressor()

    model.fit(
        X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        max_epochs=max_epochs,
        patience=100,
        batch_size=optimal_batch_size, 
        virtual_batch_size=optimal_batch_size/2,
        num_workers=0,
        drop_last=True)
    
    df['PRED'] = model.predict(tensor(df[feature_columns].values))

    #Now make the multi-step forecast
    if len(lag_values) > 0:
        forecast_df = pd.DataFrame(forecast_data, columns = forecast_column_names)
        
        for step in range(len(forecast_df)):
            station_id = df['STATION_ID'][-1:].values[0]
            future_date = (pd.to_datetime(df.iloc[-1]['DATE'])+timedelta(days=1)).strftime('%Y-%m-%d')
            lags=[df.shift(lag-1).iloc[-1]['COUNT'] for lag in lag_values]
            forecast=forecast_df.loc[forecast_df['DATE']==future_date]
            forecast=forecast.drop(labels='DATE', axis=1).values.tolist()[0]
            features=[*lags, *forecast]
            pred=round(model.predict(np.array([features]))[0][0])
            row=[future_date, station_id, pred, *features, pred]
            df.loc[len(df)]=row
    
    explain_df = pd.DataFrame(model.explain(df[feature_columns].astype(float).values)[0], 
                         columns = feature_columns).add_prefix('EXPL_').round(2)
    df = pd.concat([df.set_index('DATE').reset_index(), explain_df], axis=1)
   
    return [df.values.tolist(), df.columns.tolist()]

The Snowpark server-side Anaconda runtime has a large [list of Python modules included](https://docs.snowflake.com/en/LIMITEDACCESS/udf-python-packages.html#list-of-the-third-party-packages-from-anaconda) for our UDF.  However, the data scientist built this code based on pytorch-tabnet which is not currently in the Snowpark distribution.
  
  We can simply add [pytorch_tabnet](https://github.com/dreamquark-ai/tabnet), as well as our own team's python code, as import dependencies.

In [ ]:
from dags.station_train_predict import station_train_predict_func
import os 

#We can add dependencies from locally installed directories
#source_dir = os.environ['CONDA_PREFIX']+'/lib/python3.8/site-packages/'

model_stage_name = 'model_stage'
_ = session.sql('CREATE STAGE IF NOT EXISTS '+str(model_stage_name)).collect()

session.clear_packages()
session.clear_imports()
session.add_packages(["pandas==1.3.5", "pytorch", "scipy", "scikit-learn"])
session.add_import('./include/pytorch_tabnet.zip')
session.add_import('dags')

station_train_predict_udf = session.udf.register(station_train_predict_func, 
                                              name="station_train_predict_udf",
                                              is_permanent=True,
                                              stage_location='@'+str(model_stage_name), 
                                              replace=True)

### 4. Test the training/inference pipeline and prediction output.

We will create an array aggregation to feed the training data to our UDF.

In [ ]:
%%time

from dags.feature_engineering import generate_features

import snowflake.snowpark as snp
from snowflake.snowpark import functions as F 

station_id = '519'
target_column = 'COUNT'
lag_values=[1,7,365]
forecast_days=30
start_date='2020-03-01'

snowdf = session.table(state_dict['trips_table_name']).filter(F.col('START_STATION_ID') == station_id)

historical_df = generate_features(session=session, 
                                 input_df=snowdf, 
                                 holiday_table_name=state_dict['holiday_table_name'], 
                                 weather_table_name=state_dict['weather_table_name'])

forecast_df = create_forecast_df(session=session, 
                                 holiday_table_name=state_dict['holiday_table_name'], 
                                 weather_table_name=state_dict['weather_table_name'], 
                                 start_date=start_date, 
                                 steps=forecast_days)

#Until UDTF we need to do array stuffing on these two dataframes
historical_column_names = F.array_construct(*[F.lit(x) for x in historical_df.columns])
forecast_column_names = F.array_construct(*[F.lit(x) for x in forecast_df.columns])
lag_values_array = F.array_construct(*[F.lit(x) for x in lag_values])

historical_df = historical_df.select(F.array_agg(F.array_construct(F.col('*'))).alias('HISTORICAL_DATA'))
forecast_df = forecast_df.select(F.array_agg(F.array_construct(F.col('*'))).alias('FORECAST_DATA'))

cutpoint=365
max_epochs = 100

output_df = historical_df.join(forecast_df)\
                         .select(F.call_udf('station_train_predict_udf', 
                                            F.col('HISTORICAL_DATA'),
                                            F.lit(historical_column_names), 
                                            F.lit(target_column),
                                            F.lit(cutpoint), 
                                            F.lit(max_epochs),
                                            F.col('FORECAST_DATA'),
                                            F.lit(forecast_column_names),
                                            F.lit(lag_values_array))).collect()

In [ ]:
import pandas as pd
import ast
df = pd.DataFrame(data = ast.literal_eval(output_df[0][0])[0], 
                  columns = ast.literal_eval(output_df[0][0])[1])

df['DATE'] = pd.to_datetime(df['DATE']).dt.date
df.tail()

In [ ]:
def plot(df, x_lab:str, y_true_lab:str, y_pred_lab:str):
    plt.figure(figsize=(15, 8))
    df = pd.melt(df, id_vars=[x_lab], value_vars=[y_true_lab, y_pred_lab])
    ax = sns.lineplot(x=x_lab, y='value', hue='variable', data=df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
   
plot(df, 'DATE', 'COUNT', 'PRED')

We will end by consolidating the functions we created.

In [ ]:
# %%writefile citibike_ml/mlops_pipeline.py

# def materialize_holiday_table(session, start_date, end_date, holiday_table_name) -> str:
#     from citibike_ml.feature_engineering import generate_holiday_df
#     from datetime import datetime

#     holiday_df = generate_holiday_df(session=session, start_date=start_date, end_date=datetime.now())
#     holiday_df.write.mode('overwrite').saveAsTable(holiday_table_name)
    
#     return holiday_table_name

# def materialize_weather_table(session, start_date, end_date, weather_table_name) -> str:
#     from citibike_ml.feature_engineering import generate_weather_df
#     from datetime import datetime

#     weather_df = generate_weather_df(session=session, start_date=start_date, end_date=datetime.now())
#     weather_df.write.mode('overwrite').saveAsTable(weather_table_name)
    
#     return weather_table_name

# def deploy_pred_train_udf(session, function_name, model_stage_name) -> str:
#     from citibike_ml.station_train_predict import station_train_predict_func

#     dep = 'pytorch_tabnet.zip'
#     source_dir = './include/'

#     session.clearImports()
#     session.addImport(source_dir+dep)
#     session.addImport('citibike_ml')

#     station_train_predict_udf = session.udf.register(station_train_predict_func, 
#                                                   name="station_train_predict_udf",
#                                                   is_permanent=True,
#                                                   stage_location='@'+str(model_stage_name), 
#                                                   replace=True)
#     return station_train_predict_udf.name


# def generate_feature_views(session, 
#                            clone_table_name, 
#                            feature_view_name, 
#                            holiday_table_name, 
#                            weather_table_name, 
#                            target_column, 
#                            top_n) -> list:
#     from citibike_ml.feature_engineering import generate_features
#     from snowflake.snowpark import functions as F

#     feature_view_names = list()
    
#     top_n_station_ids = session.table(clone_table_name).filter(F.col('START_STATION_ID').is_not_null()) \
#                                                        .groupBy('START_STATION_ID') \
#                                                        .count() \
#                                                        .sort('COUNT', ascending=False) \
#                                                        .limit(top_n) \
#                                                        .collect()
#     top_n_station_ids = [stations['START_STATION_ID'] for stations in top_n_station_ids]

#     for station in top_n_station_ids:
#         feature_df = generate_features(session=session, 
#                                        input_df=session.table(clone_table_name)\
#                                                        .filter(F.col('START_STATION_ID') == station), 
#                                        holiday_table_name=holiday_table_name, 
#                                        weather_table_name=weather_table_name)

#         input_columns_str = str(' ').join(feature_df.columns).replace('\"', "")

#         feature_df = feature_df.select(F.array_agg(F.array_construct(F.col('*'))).alias('input_data'), 
#                                        F.lit(station).alias('station_id'),
#                                        F.lit(input_columns_str).alias('input_column_names'),
#                                        F.lit(target_column).alias('target_column'))  

#         station_feature_view_name = feature_view_name.replace('<station_id>', station)
#         feature_df.createOrReplaceView(station_feature_view_name)
#         feature_view_names.append(station_feature_view_name)

#     return feature_view_names


# def train_predict_feature_views(session, station_train_pred_udf_name, feature_view_names, pred_table_name) -> str:
#     from snowflake.snowpark import functions as F
#     import pandas as pd
#     import ast
    
#     cutpoint=365
#     max_epochs=1000
    
#     for view in feature_view_names:
#         feature_df = session.table(view)
#         output_df = feature_df.select(F.call_udf(station_train_pred_udf_name, 
#                                                  'INPUT_DATA', 
#                                                  'INPUT_COLUMN_NAMES', 
#                                                  'TARGET_COLUMN', 
#                                                  F.lit(cutpoint), 
#                                                  F.lit(max_epochs))).collect()

#         df = pd.DataFrame(data = ast.literal_eval(output_df[0][0])[0], 
#                       columns = ast.literal_eval(output_df[0][0])[1])

#         df['DATE'] = pd.to_datetime(df['DATE']).dt.date
#         df['STATION_ID'] = feature_df.select('STATION_ID').collect()[0][0]

#         output_df = session.createDataFrame(df).write.saveAsTable(pred_table_name)
    
#     return pred_table_name

# def create_forecast_df(session, holiday_table_name:str, weather_table_name:str, start_date:str, steps:int):
#     from dags.feature_engineering import generate_holiday_df, generate_weather_df
#     from datetime import timedelta, datetime
#     #import snowflake.snowpark as snp
#     from snowflake.snowpark import functions as F 
    
#     start_date = datetime.strptime(start_date, '%Y-%m-%d')
#     end_date = start_date+timedelta(days=steps)

#     #check if it tables already materialized, otherwise generate DF
#     holiday_df = session.table(holiday_table_name)
#     try: 
#         _ = holiday_df.columns()
#     except:
#         holiday_df = generate_holiday_df(session, holiday_table_name)
        
#     weather_df = session.table(weather_table_name)[['DATE','TEMP']]
#     try: 
#         _ = weather_df.columns()
#     except:
#         weather_df = generate_weather_df(session, weather_table_name)[['DATE','TEMP']]
        
#     forecast_df = holiday_df.join(weather_df, 'DATE', join_type='right')\
#                             .na.fill({'HOLIDAY':0})\
#                             .filter((F.col('DATE') >= start_date) &\
#                                     (F.col('DATE') <= end_date))\
#                             .sort('DATE', ascending=True)
#     return forecast_df

In [ ]:
session.close()